In [93]:
#imports
import pandas as pd
import numpy as np
import math

In [91]:
#data imports
regions = pd.read_csv(r'risk_data.csv', sep=";", usecols=[0])
risk_data = pd.read_csv(r'risk_data.csv', sep=";", index_col=0)
asset_values = pd.read_csv(r'asset_values.csv', sep = ";", index_col=0)
influences = pd.read_csv(r'influences.csv', sep=";", index_col = 0)
base_data = pd.read_csv(r'base_data.csv', sep=";", index_col=0)

In [77]:
agg_risk = risk_data.sum(axis=1)

In [85]:
#risk per capita
rpk = agg_risk/base_data["pop"]
yearly_wage = base_data["sim_wage"]*52


In [105]:
#transfer payment method 1
transfer_payment = 0.2*sum(yearly_wage)*(rpk/sum(rpk))-0.2*yearly_wage

In [103]:
#initializing base states - I think not really necessary since we used it to calculate b in the Excel. Only for welfare calc
v_base = base_data["b_i"]*yearly_wage/rpk
PI_base = v_base/sum(v_base)
H_base = PI_base*sum(base_data["pop"])
A_base = np.empty(len(agg_risk), dtype="float")
for i in range(len(agg_risk)):
    A_base[i] = math.pow(H_base[i], (base_data["prod"][i]/100))

In [110]:
#time_step_0
v_0=(base_data["b_i"]*(yearly_wage+transfer_payment)/rpk)
PI_0=v_0/sum(v_0)
H_0=PI_0*sum(base_data["pop"])
pop_dens_0=H_0/base_data["area"]
#wage level adaption based on population density --> Linear regression on weekly wage shows beta_0 = 598.74 and beta_1 = 0.0359
wage_1 = (0.0359*pop_dens_0+598.74)*52

#time_step_1
rpk_1 = agg_risk/H_0
v_1=(base_data["b_i"]*(wage_1+transfer_payment)/rpk_1)
PI_1 = v_1/sum(v_1)
H_1 = PI_1*sum(base_data["pop"])
A_1 = np.empty(len(agg_risk), dtype="float")
for i in range(len(agg_risk)):
    A_1[i] = math.pow(H_1[i], (base_data["prod"][i]/100)) #assumption: Productivity is based on regional factors and does not change


In [115]:
welfare_increase = sum(A_1)-sum(A_base)
inc_perc = 100*welfare_reduction/sum(A_base)

In [116]:
print("The increase in welfare with the given transfer policy is %s which is an increase of %s percent." % (welfare_increase, inc_perc))

The increase in welfare with the given transfer policy is -17017605825.303848 which is an increase of -37.88639120198049 percent.
